# Coupon Code Generator

We're implementing a coupon code redemption system at work. Since I was in a hurry I purchased a block of codes from a commecial service, but I thought recreating it would be a fun programming exercise.

In [1]:
import string
import random
import pandas as pd
import re

def generate(pattern, disallow='O0ILil'):
    '''
    Generate a coupon code based on a given pattern.
    Pattern is generated character-by-character so we can
    extend the pattern options fairly easily.
    X = Uppercase (A, B, C, ...)
    x = Lowercase (a, b, c, ...)
    9 = Digits (0, 1, 2, ...)
    A = Uppercase + Digits (A, B, .. + 0, 1 ..)
    a = Lowercase + Digits (a, b, .. + 0, 1 ..)
    # = Special characters (@, #, $, %, ^, &, *)
    v = Vowels (a, e, i, ...)
    V = Uppercase Voels (A, E, I, ...)
    c = Consonants (b, c, d, ...)
    C = Uppercase Consonants (B, C, D, ...)
    ? = Random from all characters above
    - = - 
    / = Escape character 
    Any character not on this list will be used as-is
    '''
    uppercase = string.ascii_uppercase
    lowercase = string.ascii_lowercase
    digits = ''.join(map(str, [i for i in range(10)]))
    upperdigits = uppercase + digits
    lowerdigits = lowercase + digits
    special = '@#$%^&*'
    vowels = 'aeiou'
    uppervowels = 'AEIOU'
    consonants = 'bcdfghjklmnpqrstvwxz'
    upperconsonants = 'BCDFGHJKLMNPQRSTVWXZ'
    everything = uppercase + lowercase + digits + special
    
    # Remove any disallowed characters such as the always-confusing 0/O
    for char in disallow:
        uppercase = uppercase.replace(char, '')
        lowercase = lowercase.replace(char, '')
        digits = digits.replace(char, '')
        upperdigits = upperdigits.replace(char, '')
        lowerdigits = lowerdigits.replace(char, '')
        vowels = vowels.replace(char, '')
        uppervowels = uppervowels.replace(char, '')
        consonants = consonants.replace(char, '')
        upperconsonants = upperconsonants.replace(char, '')
        everything = everything.replace(char, '')
    
    escape = 0
    output = ''
    
    for option in pattern:
        if escape == 1:
            output += option
            escape = 0
        elif option == 'X':
            output += random.choice(uppercase)
        elif option == 'x':
            output += random.choice(lowercase)
        elif option == '9':
            output += random.choice(digits)
        elif option == 'A':
            output += random.choice(upperdigits)
        elif option == 'a':
            output += random.choice(lowerdigits)
        elif option == '#':
            output += random.choice(special)
        elif option == 'v':
            output += random.choice(vowels)
        elif option == 'V':
            output += random.choice(uppervowels)
        elif option == 'c':
            output += random.choice(consonants)
        elif option == 'C':
            output += random.choice(upperconsonants)
        elif option == '?':
            output += random.choice(everything)
        elif option == '-':
            output += '-'
        elif option == '/':
            escape = 1
        else:
            output += option
    return output

print(generate('Xx9A-a#vV-cC?/9/C-1234'))

Qw8F-1*oE-zCY9C-1234


In [2]:
def save(items, path='codes.csv'):
    '''
    Save an iterable to a csv
    '''
    data = {'codes': items}
    df = pd.DataFrame(data, columns = ['codes'])
    df.to_csv('codes.csv')

In [3]:
def generateAndSave(pattern, disallow='O0ILil', quantity='100'):
    '''
    Generate many unique codes and save them to a file.
    '''
    
    # Generate codes, sort them, strip out duplicates
    file = [generate(pattern, disallow) for i in range(quantity)]
    file.sort()
    file = [x for ind, x in enumerate(file) if ind < len(file)-1 and x != file[ind+1]]
       
    # Generate the last few codes if we had to remove any duplicates
    i = len(file)
    while i < quantity:
        code = generate(pattern, disallow)
        if code not in file:
            file.append(code)
            i += 1
    
    # If we're using a subset of the codes, we might not want them
    # to share several characters at the start. TODO: make optional
    random.shuffle(file)
    
    save(file)
    print('File saved!')

In [4]:
generateAndSave('Xx9A-a#vV-cC?/9/C-1234', quantity=100)

File saved!


In [5]:
def validate(code, pattern, disallow='O0ILil'):
    '''
    Given a code and the rules for code generation, determine whether the code is valid.
    TODO: Let this loop through a list of codes without generating the regex over and over.
    '''
    
    escape = 0
    regex = ''
    special = '.^$*+?{}[]\|()'
    
    # Check for disallowed characters here rather than make the regex crazy
    for char in disallow:
        if char in code:
            print("Disallowed character!")
            return False
    
    # This pretty much repeats the logic from generate, but generates a regular expresison.
    for char in pattern:
        if char in special and not '?':
            regex += '\\'
        
        if escape == 1:
            regex += char
            escape = 0
        elif char == 'X':
            regex += '[A-Z]'
        elif char == 'x':
            regex += '[a-z]'
        elif char == '9':
            regex += '[0-9]'
        elif char == 'A':
            regex += '[A-Z0-9]'
        elif char == 'a':
            regex += '[a-z0-9]'
        elif char == '#':
            regex += '[@#\$%\^&\*]'
        elif char == 'v':
            regex += '[aeiou]'
        elif char == 'V':
            regex += '[AEIOU]'
        elif char == 'c':
            regex += '[bcdfghjklmnpqrstvwxz]'
        elif char == 'C':
            regex += '[BCDFGHJKLMNPQRSTVWXZ]'
        elif char == '?':
            regex += '\S'
        elif char == '/':
            escape = 1
        else:
            regex += char
    
    print('Code: %s' % code)
    print('Pattern: %s' % pattern)
    print('Regex: %s' % regex)
    regex = re.compile(regex)
    match = regex.match(code)
    
    if match: 
        print("Match!")
        return True 
    else: 
        print("No Match")
        return False

In [6]:
pattern = 'Xx9A-a#vV-cC?/9/C-1234'
code = generate(pattern)

pattern2 = 'Xx9A-a#vV-cC?/9/C-5678'

validate(code, pattern)
validate(code, pattern2)

Code: Yw83-n&aE-mZf9C-1234
Pattern: Xx9A-a#vV-cC?/9/C-1234
Regex: [A-Z][a-z][0-9][A-Z0-9]-[a-z0-9][@#\$%\^&\*][aeiou][AEIOU]-[bcdfghjklmnpqrstvwxz][BCDFGHJKLMNPQRSTVWXZ]\S9C-1234
Match!
Code: Yw83-n&aE-mZf9C-1234
Pattern: Xx9A-a#vV-cC?/9/C-5678
Regex: [A-Z][a-z][0-9][A-Z0-9]-[a-z0-9][@#\$%\^&\*][aeiou][AEIOU]-[bcdfghjklmnpqrstvwxz][BCDFGHJKLMNPQRSTVWXZ]\S9C-5678
No Match


False